##  Structure générale du notebook `01_cleaning_EDA.ipynb`

Ce notebook est organisé en plusieurs sections logiques afin de conduire une **analyse exploratoire complète et structurée** du jeu de données `df`.

### **1️⃣ Importation et chargement des données**
Objectif : importer les bibliothèques nécessaires et charger le DataFrame `df` depuis le notebook précédent (`00_setup_data_load.ipynb`) ou depuis le fichier `.parquet`.

> Vérifier que le chargement fonctionne et afficher les premières lignes pour un aperçu global.

###  Vérification de la qualité des données**
Objectif : évaluer la **qualité et la cohérence** des données.

- Identifier les valeurs manquantes (`NaN`), doublons et incohérences.  
- Examiner les types de colonnes et la présence de valeurs extrêmes.  
- Calculer des statistiques descriptives de base.

###  Nettoyage et préparation (si nécessaire)**
Objectif : appliquer des **corrections légères** avant d’analyser.

- Supprimer les enregistrements incomplets ou aberrants.  
- Gérer les valeurs manquantes (remplissage ou suppression).  
- Restreindre les délais à une plage raisonnable (ex. 0–60 jours).


### Statistiques descriptives et agrégées**
Objectif : obtenir une vue d’ensemble des **indicateurs clés** :

- Taux de retard global (%).  
- Délai moyen de livraison.  
- Prix et poids moyens des produits.  
- Répartition des paiements et du fret.


### Analyse de corrélation**
Objectif : comprendre les **relations entre variables** numériques.

- Calculer la matrice de corrélation (Spearman).  
- Identifier les variables fortement liées au retard (`delivered_late`).  
- Visualiser les corrélations avec une *heatmap* interactive.

### Analyse statistique (tests de différences)**
Objectif : vérifier si certaines variables présentent des **différences significatives** entre :
- commandes livrées à l’heure, et  
- commandes livrées en retard.

> Exemple : test de Mann–Whitney pour comparer le poids moyen ou le montant du fret.

###  Visualisations exploratoires**
Objectif : représenter graphiquement les **tendances clés** du dataset.

- Histogrammes : distribution des délais (`days_to_deliver`).  
- Boxplots : comparaison des délais et montants selon le statut de livraison.  
- Cartes et barplots : retards par État, par catégorie produit ou par mois.  
- Graphiques interactifs (Plotly).

### Synthèse et insights business**
Objectif : extraire les **enseignements clés** de l’analyse exploratoire.

- Résumer les principales observations (corrélations, anomalies, tendances).  
- Identifier les **leviers d’amélioration logistique** (transport, vendeur, catégories).  
- Préparer la transition vers la phase de modélisation (`02_model_evaluation.ipynb`).

 **Conclusion :**
Ce notebook te permettra de transformer ton jeu de données en **connaissance utile**, et d’identifier les **facteurs influençant le retard de livraison** — étape essentielle avant toute modélisation prédictive.


In [6]:
# Importation des bibliothèques & chargement du DataFrame

# --- Import des librairies principales ---
import os
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
from scipy import stats


# --- Chargement du DataFrame ---
if DATA_PATH.exists():
    df = pd.read_parquet(DATA_PATH)
    print("✅ Fichier Parquet chargé :", df.shape)
else:

    DATA = Path("../data")
    orders = pd.read_csv(DATA/"olist_orders_dataset.csv",
                         parse_dates=[
                             "order_purchase_timestamp", "order_approved_at",
                             "order_delivered_carrier_date", "order_delivered_customer_date",
                             "order_estimated_delivery_date"
                         ])
    items = pd.read_csv(DATA/"olist_order_items_dataset.csv")
    products = pd.read_csv(DATA/"olist_products_dataset.csv")
    cust = pd.read_csv(DATA/"olist_customers_dataset.csv")
    sellers = pd.read_csv(DATA/"olist_sellers_dataset.csv")
    pay = pd.read_csv(DATA/"olist_order_payments_dataset.csv")

    # --- Fusion rapide (vue simplifiée pour l'EDA) ---
    df = (
        orders
        .merge(items, on="order_id", how="left")
        .merge(products, on="product_id", how="left")
        .merge(
            pay.groupby("order_id", as_index=False)
               .agg({"payment_value": "sum", "payment_installments": "max"}),
            on="order_id", how="left"
        )
        .merge(
            cust[["customer_id","customer_city","customer_state"]],
            on="customer_id", how="left"
        )
        .merge(
            sellers[["seller_id","seller_city","seller_state"]],
            on="seller_id", how="left"
        )
    )
    print(" Données CSV fusionnées :", df.shape)

# --- Aperçu des premières lignes ---
display(df.head(3))

# --- Vérification des colonnes principales ---
print("\nColonnes disponibles :")
print(df.columns.tolist()[:15], "...")


✅ Données CSV fusionnées : (113425, 28)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_value,payment_installments,customer_city,customer_state,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1.0,87285b34884572647811a353c7ac498a,...,500.0,19.0,8.0,13.0,38.71,1.0,sao paulo,SP,maua,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1.0,595fac2a385ac33a80bd5114aec74eb8,...,400.0,19.0,13.0,19.0,141.46,1.0,barreiras,BA,belo horizonte,SP
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1.0,aa4383b373c6aca5d8797843e5594415,...,420.0,24.0,19.0,21.0,179.12,3.0,vianopolis,GO,guariba,SP



Colonnes disponibles :
['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'product_category_name'] ...


# Vérification de la qualité des données

In [8]:
# --- 2.1 Structure générale du DataFrame ---
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113425 entries, 0 to 113424
Data columns (total 28 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       113425 non-null  object        
 1   customer_id                    113425 non-null  object        
 2   order_status                   113425 non-null  object        
 3   order_purchase_timestamp       113425 non-null  datetime64[ns]
 4   order_approved_at              113264 non-null  datetime64[ns]
 5   order_delivered_carrier_date   111457 non-null  datetime64[ns]
 6   order_delivered_customer_date  110196 non-null  datetime64[ns]
 7   order_estimated_delivery_date  113425 non-null  datetime64[ns]
 8   order_item_id                  112650 non-null  float64       
 9   product_id                     112650 non-null  object        
 10  seller_id                      112650 non-null  object        
 11  

In [9]:
# --- 2.2 Aperçu des premières lignes ---
display(df.head(5))

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_value,payment_installments,customer_city,customer_state,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1.0,87285b34884572647811a353c7ac498a,...,500.0,19.0,8.0,13.0,38.71,1.0,sao paulo,SP,maua,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1.0,595fac2a385ac33a80bd5114aec74eb8,...,400.0,19.0,13.0,19.0,141.46,1.0,barreiras,BA,belo horizonte,SP
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1.0,aa4383b373c6aca5d8797843e5594415,...,420.0,24.0,19.0,21.0,179.12,3.0,vianopolis,GO,guariba,SP
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,...,450.0,30.0,10.0,20.0,72.20,1.0,sao goncalo do amarante,RN,belo horizonte,MG
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.0,65266b2da20d04dbe00c5c2d3bb7859e,...,250.0,51.0,15.0,15.0,28.62,1.0,santo andre,SP,mogi das cruzes,SP


In [10]:
# --- 2.3 Vérification des valeurs manquantes ---
missing_values = (
    df.isna()
      .sum()
      .sort_values(ascending=False)
      .to_frame("nb_valeurs_manquantes")
)
missing_values["pourcentage"] = 100 * missing_values["nb_valeurs_manquantes"] / len(df)
display(missing_values.head(10))

,nb_valeurs_manquantes,pourcentage
order_delivered_customer_date,3229,2.846815
product_photos_qty,2378,2.096540
product_category_name,2378,2.096540
product_name_lenght,2378,2.096540
product_description_lenght,2378,2.096540
order_delivered_carrier_date,1968,1.735067
product_length_cm,793,0.699140
product_weight_g,793,0.699140
product_height_cm,793,0.699140
product_width_cm,793,0.699140


In [11]:
# --- 2.4 Vérification des doublons ---
nb_duplicates = df.duplicated().sum()
print(f"🧩 Nombre de doublons détectés : {nb_duplicates}")

🧩 Nombre de doublons détectés : 0


In [12]:
# --- 2.5 Statistiques descriptives globales ---
desc = df.describe(include="all").T
display(desc.head(10))

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
order_id,113425,99441,8272b63d03f5f79c56e9e4120aec44ef,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
customer_id,113425,99441,fc3d1daec319d62d49bfb5e1f83123e9,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN
order_status,113425,8,delivered,110197,NaN,NaN,NaN,NaN,NaN,NaN,NaN
order_purchase_timestamp,113425,NaN,NaN,NaN,2017-12-31 12:25:41.686480128,2016-09-04 21:15:19,2017-09-13 11:05:49,2018-01-19 10:37:45,2018-05-04 14:22:16,2018-10-17 17:30:18,NaN
order_approved_at,113264,NaN,NaN,NaN,2017-12-31 22:40:56.664121088,2016-09-15 12:16:38,2017-09-13 20:25:10.500000,2018-01-19 16:59:52.500000,2018-05-04 18:31:38.500000,2018-09-03 17:40:06,NaN
order_delivered_carrier_date,111457,NaN,NaN,NaN,2018-01-05 02:17:27.942515968,2016-10-08 10:34:01,2017-09-18 20:37:00,2018-01-24 18:44:33,2018-05-08 13:20:00,2018-09-11 19:48:28,NaN
order_delivered_customer_date,110196,NaN,NaN,NaN,2018-01-14 13:25:24.023939328,2016-10-11 13:46:32,2017-09-26 20:09:44.500000,2018-02-02 20:57:23,2018-05-15 20:09:21.500000,2018-10-17 13:22:46,NaN
order_estimated_delivery_date,113425,NaN,NaN,NaN,2018-01-24 08:22:34.875909120,2016-09-30 00:00:00,2017-10-04 00:00:00,2018-02-15 00:00:00,2018-05-25 00:00:00,2018-11-12 00:00:00,NaN
order_item_id,112650.0,NaN,NaN,NaN,1.197834,1.0,1.0,1.0,1.0,21.0,0.705124
product_id,112650,32951,aca2eb7d00ea1a7b8ebd4e68314663af,527,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:

# --- 2.6 Vérification des valeurs extrêmes sur certaines variables numériques ---
num_cols = ["price", "freight_value", "product_weight_g", "payment_value", "days_to_deliver"]
for col in num_cols:
    if col in df.columns:
        q1, q3 = df[col].quantile([0.25, 0.75])
        iqr = q3 - q1
        low, high = q1 - 1.5 * iqr, q3 + 1.5 * iqr
        outliers = df[(df[col] < low) | (df[col] > high)].shape[0]
        print(f" {col}: {outliers} valeurs extrêmes détectées (hors [{low:.1f}, {high:.1f}])")


 price: 8427 valeurs extrêmes détectées (hors [-102.6, 277.4])
 freight_value: 12134 valeurs extrêmes détectées (hors [1.0, 33.3])
 product_weight_g: 15807 valeurs extrêmes détectées (hors [-1950.0, 4050.0])
 payment_value: 9285 valeurs extrêmes détectées (hors [-128.8, 389.9])


# Nettoyage et préparation des données


In [17]:
# --- 3.1 Suppression des doublons (si existants) ---
nb_duplicates = df.duplicated().sum()
if nb_duplicates > 0:
    df = df.drop_duplicates()
    print(f" {nb_duplicates} doublons supprimés.")
else:
    print(" Aucun doublon détecté.")

# --- 3.2 Suppression des lignes sans date de livraison effective ---
if "order_delivered_customer_date" in df.columns:
    before = len(df)
    df = df[df["order_delivered_customer_date"].notna()]
    print(f" Lignes sans date de livraison supprimées : {before - len(df)}")

 Aucun doublon détecté.
 Lignes sans date de livraison supprimées : 0


In [18]:
# --- 3.3 Suppression des valeurs aberrantes sur les délais de livraison ---
if "days_to_deliver" in df.columns:
    before = len(df)
    df = df[df["days_to_deliver"].between(0, 60)]  # 0 à 60 jours
    print(f" Commandes avec délais aberrants supprimées : {before - len(df)}")

In [21]:
# --- 3.4 Traitement des valeurs manquantes sur les variables numériques ---
num_cols = ["price", "freight_value", "product_weight_g", "payment_value"]
for col in num_cols:
    if col in df.columns:
        missing = df[col].isna().sum()
        if missing > 0:
            median_value = df[col].median()
            df[col].fillna(median_value, inplace=True)
            print(f" {missing} valeurs manquantes remplies par la médiane pour '{col}'.")

In [22]:
# --- 3.5 Vérification des NaN restants ---
remaining_missing = df.isna().sum().sum()
print(f"🔍 Nombre total de valeurs manquantes restantes : {remaining_missing}")

# --- 3.6 Nettoyage des valeurs négatives (si anomalies) ---
for col in ["price", "freight_value", "payment_value"]:
    if col in df.columns:
        neg = df[df[col] < 0].shape[0]
        if neg > 0:
            df.loc[df[col] < 0, col] = np.nan
            print(f"⚠️ {neg} valeurs négatives détectées dans '{col}', remplacées par NaN.")
            df[col].fillna(df[col].median(), inplace=True)
print("Valeurs négatives corrigées.")

🔍 Nombre total de valeurs manquantes restantes : 6221
Valeurs négatives corrigées.


In [23]:
# --- 3.7 Réinitialisation de l’index ---
df.reset_index(drop=True, inplace=True)

print("\n Nettoyage terminé. Le DataFrame est prêt pour l'analyse exploratoire.")
print(f"Forme finale du DataFrame : {df.shape}")


 Nettoyage terminé. Le DataFrame est prêt pour l'analyse exploratoire.
Forme finale du DataFrame : (110196, 28)


# 4 Statistiques descriptives et indicateurs clés


In [25]:
# --- 4.1 Dimensions du DataFrame ---
nb_lignes, nb_colonnes = df.shape
print(f" Nombre d'observations : {nb_lignes:,}")
print(f" Nombre de variables : {nb_colonnes}")

 Nombre d'observations : 110,196
 Nombre de variables : 28


In [26]:
# --- 4.2 Taux de retard global ---
if "delivered_late" in df.columns:
    late_rate = df["delivered_late"].mean()
    print(f"Taux de retard global : {late_rate:.2%}")
else:
    print("Variable 'delivered_late' non trouvée.")

Variable 'delivered_late' non trouvée.


In [27]:
# --- 4.3 Délai moyen de livraison ---
if "days_to_deliver" in df.columns:
    avg_days = df["days_to_deliver"].mean()
    median_days = df["days_to_deliver"].median()
    print(f"⏱️ Délai moyen de livraison : {avg_days:.1f} jours (médiane : {median_days:.1f})")


In [28]:
# --- 4.4 Prix moyen et distribution du fret ---
if "price" in df.columns and "freight_value" in df.columns:
    print(f" Prix moyen produit : {df['price'].mean():.2f} R$")
    print(f" Coût moyen de fret : {df['freight_value'].mean():.2f} R$")


 Prix moyen produit : 119.98 R$
 Coût moyen de fret : 19.95 R$


In [30]:
# --- 4.5 Montant total payé et nombre moyen d’échéances ---
if "payment_value" in df.columns and "payment_installments" in df.columns:
    print(f"Montant moyen payé : {df['payment_value'].mean():.2f} R$")
    print(f"Nombre moyen d'échéances : {df['payment_installments'].mean():.1f}")

Montant moyen payé : 179.46 R$
Nombre moyen d'échéances : 3.0


In [31]:
# --- 4.7 États les plus représentés chez les clients ---
if "customer_state" in df.columns:
    top_states = (
        df["customer_state"]
        .value_counts()
        .head(10)
        .reset_index()
        .rename(columns={"index": "État", "customer_state": "commandes"})
    )
    print("\n Top 10 des États clients :")
    display(top_states)



 Top 10 des États clients :


,commandes,count
0,SP,46443
1,RJ,14146
2,MG,12917
3,RS,6133
4,PR,5649
5,SC,4098
6,BA,3683
7,DF,2355
8,GO,2277
9,ES,2225


In [39]:
# Vérifier la présence de la colonne 'delivered_late'
if "delivered_late" not in df.columns:
    print(" Colonne 'delivered_late' absente — création automatique.")
    if {"order_delivered_customer_date", "order_estimated_delivery_date"}.issubset(df.columns):
        df["delivered_late"] = (
            df["order_delivered_customer_date"] > df["order_estimated_delivery_date"]
        ).astype(int)
        print(" Colonne 'delivered_late' recréée avec succès.")
    else:
        raise KeyError(" Impossible de créer 'delivered_late' : colonnes de dates manquantes.")


In [ ]:
# Vérifier la présence de la colonne 'delivered_late'
if "delivered_late" not in df.columns:
    print("⚠️ Colonne 'delivered_late' absente — création automatique.")
    if {"order_delivered_customer_date", "order_estimated_delivery_date"}.issubset(df.columns):
        df["delivered_late"] = (
            df["order_delivered_customer_date"] > df["order_estimated_delivery_date"]
        ).astype(int)
        print("✅ Colonne 'delivered_late' recréée avec succès.")
    else:
        raise KeyError("Impossible de créer 'delivered_late' : colonnes de dates manquantes.")


In [32]:
# --- 4.8 Répartition du retard par mois (si dates dispo) ---
if "order_purchase_timestamp" in df.columns and "delivered_late" in df.columns:
    monthly = (
        df.groupby(df["order_purchase_timestamp"].dt.to_period("M"))["delivered_late"]
        .mean()
        .reset_index()
    )
    monthly["order_purchase_timestamp"] = monthly["order_purchase_timestamp"].astype(str)

    fig = px.line(
        monthly,
        x="order_purchase_timestamp",
        y="delivered_late",
        title=" Évolution du taux de retard au fil du temps",
        markers=True
    )
    fig.update_yaxes(title="Taux de retard", tickformat=".0%")
    fig.show()

In [42]:
monthly = (
    df.groupby(df["order_purchase_timestamp"].dt.to_period("M"))["delivered_late"]
    .mean()
    .reset_index()
)
monthly["order_purchase_timestamp"] = monthly["order_purchase_timestamp"].astype(str)

fig = px.line(
    monthly,
    x="order_purchase_timestamp",
    y="delivered_late",
    title="Évolution du taux de retard au fil du temps",
    markers=True
)
fig.update_yaxes(title="Taux de retard", tickformat=".0%")
fig.show()
